In [0]:
import datetime
import json
import os
import pprint
import random
import string
import sys
import csv
import pandas as pd
import tensorflow as tf

assert 'COLAB_TPU_ADDR' in os.environ, 'ERROR: Not connected to a TPU runtime; please see the first cell in this notebook for instructions!'
TPU_ADDRESS = 'grpc://' + os.environ['COLAB_TPU_ADDR']
print('TPU address is', TPU_ADDRESS)

from google.colab import auth
auth.authenticate_user()
with tf.Session(TPU_ADDRESS) as session:
  print('TPU devices:')
  pprint.pprint(session.list_devices())

  # Upload credentials to TPU.
  with open('/content/adc.json', 'r') as f:
    auth_info = json.load(f)
  tf.contrib.cloud.configure_gcs(session, credentials=auth_info)
  # Now credentials are set for all future sessions on this TPU.

TPU address is grpc://10.114.153.170:8470
The TensorFlow contrib module will not be included in TensorFlow 2.0.
For more information, please see:
  * https://github.com/tensorflow/community/blob/master/rfcs/20180907-contrib-sunset.md
  * https://github.com/tensorflow/addons
  * https://github.com/tensorflow/io (for I/O related ops)
If you depend on functionality not listed there, please file an issue.

TPU devices:
[_DeviceAttributes(/job:tpu_worker/replica:0/task:0/device:CPU:0, CPU, -1, 17777810768514045616),
 _DeviceAttributes(/job:tpu_worker/replica:0/task:0/device:XLA_CPU:0, XLA_CPU, 17179869184, 14096024389885144369),
 _DeviceAttributes(/job:tpu_worker/replica:0/task:0/device:TPU:0, TPU, 17179869184, 11013081560207731778),
 _DeviceAttributes(/job:tpu_worker/replica:0/task:0/device:TPU:1, TPU, 17179869184, 6538526586047348971),
 _DeviceAttributes(/job:tpu_worker/replica:0/task:0/device:TPU:2, TPU, 17179869184, 16180161425135543122),
 _DeviceAttributes(/job:tpu_worker/replica:0/tas

In [0]:
import sys
from os import listdir
!rm -rf bert_finetune
!test -d bert_adapter || git clone https://github.com/junhahyung/adapter-bert bert_adapter
if not 'bert_adapter' in sys.path:
  sys.path += ['bert_adapter']
print(listdir("bert_adapter"))
# import python modules defined by BERT
import modeling
import optimization
import run_classifier
import run_classifier_with_tfhub
import tokenization

# import tfhub 
import tensorflow_hub as hub

Cloning into 'bert_adapter'...
remote: Enumerating objects: 42, done.
remote: Counting objects: 100% (42/42), done.
remote: Compressing objects: 100% (28/28), done.
remote: Total 92 (delta 26), reused 30 (delta 14), pack-reused 50
Unpacking objects: 100% (92/92), done.
['optimization_test.py', 'LICENSE', '.git', '__init__.py', 'run_classifier_with_tfhub.py', 'modeling_test.py', 'tokenization_test.py', 'tokenization.py', 'modeling.py', 'README.md', 'run_classifier.py', 'optimization.py', 'requirements.txt']



In [0]:
# korean_sa : 5 labels / korean_sa_4 : 4 labels(without neutral)
TASK = 'korean_sa_4'
DATA = 'korean_sa'
assert TASK in ('MRPC', 'CoLA', 'korean_sa', 'korean_sa_4'), 'Undefined task'

BUCKET = 'mbertfinetune' #@param {type:"string"}
assert BUCKET, 'Must specify an existing GCS bucket name'
TASK_DATA_DIR = 'gs://{}/data/{}'.format(BUCKET, DATA)
print('***** Task data directory: {} *****'.format(TASK_DATA_DIR))
print(tf.gfile.ListDirectory(TASK_DATA_DIR))
OUTPUT_DIR = 'gs://{}/bert-adapter-tfhub/models/{}'.format(BUCKET, TASK)
tf.gfile.MakeDirs(OUTPUT_DIR)
print('***** Model output directory: {} *****'.format(OUTPUT_DIR))

BERT_MODEL = 'multi_cased_L-12_H-768_A-12'
BERT_MODEL_HUB = 'https://tfhub.dev/google/bert_' + BERT_MODEL + '/1'
print(BERT_MODEL_HUB)

CKPT_DIR = 'gs://{}/{}/adapter/bert_model.ckpt'.format(BUCKET, BERT_MODEL)
CONFIG_DIR = 'gs://{}/{}/bert_config.json'.format(BUCKET, BERT_MODEL)


***** Task data directory: gs://mbertfinetune/data/korean_sa *****
['final_single_plus_multi_plus_naver_test.csv', 'final_single_plus_multi_plus_naver_train.csv', 'final_single_plus_naver_test.csv', 'final_single_plus_naver_train.csv', 'final_test_without_neutral.csv', 'final_train_without_neutral.csv', 'final_with_paraphrase.csv', 'noncleansing_final_single_plus_naver_test_original.csv', 'noncleansing_final_single_plus_naver_train_original.csv']
***** Model output directory: gs://mbertfinetune/bert-adapter-tfhub/models/korean_sa_4 *****
https://tfhub.dev/google/bert_multi_cased_L-12_H-768_A-12/1


In [0]:
tokenizer = run_classifier_with_tfhub.create_tokenizer_from_hub_module(BERT_MODEL_HUB)
tokenizer.tokenize("한국어는 잘 안되는 것 같아요.")

INFO:tensorflow:Saver not created because there are no variables in the graph to restore


INFO:tensorflow:Saver not created because there are no variables in the graph to restore


['한국', '##어는', '잘', '안', '##되는', '것', '같', '##아', '##요', '.']

In [0]:
TRAIN_BATCH_SIZE = 32
EVAL_BATCH_SIZE = 8
PREDICT_BATCH_SIZE = 8
LEARNING_RATE = 3e-4 
NUM_TRAIN_EPOCHS = 3 
MAX_SEQ_LENGTH = 128
# Warmup is a period of time where hte learning rate 
# is small and gradually increases--usually helps training.
WARMUP_PROPORTION = 0.1
# Model configs
SAVE_CHECKPOINTS_STEPS = 1000
SAVE_SUMMARY_STEPS = 500

''' layer_wise_lr applies different learning rates on different layers
    ex) layer_wise_lr = (True, 0.3) => init_lr on top layer, init_lr * 0.3 on second to top layer ..
'''
# layer_wise_lr = (True, 0.3)
# layer_wise_lr = None 

processors = {
  "cola": run_classifier.ColaProcessor,
  "mnli": run_classifier.MnliProcessor,
  "mrpc": run_classifier.MrpcProcessor,
  "korean_sa": run_classifier.KsaProcessor,
  "korean_sa_4": run_classifier.KsaProcessor_4
}
processor = processors[TASK.lower()]()
label_list = processor.get_labels()

# Compute number of train and warmup steps from batch size
train_examples = processor.get_train_examples(TASK_DATA_DIR)
num_train_steps = int(len(train_examples) / TRAIN_BATCH_SIZE * NUM_TRAIN_EPOCHS)

print(train_examples)

num_warmup_steps = int(num_train_steps * WARMUP_PROPORTION)

tpu_cluster_resolver = tf.contrib.cluster_resolver.TPUClusterResolver(TPU_ADDRESS)
NUM_TPU_CORES = 8
ITERATIONS_PER_LOOP = 1000 

def get_run_config(output_dir):  
  return tf.contrib.tpu.RunConfig(    
      cluster=tpu_cluster_resolver,    
      model_dir=output_dir,    
      save_checkpoints_steps=SAVE_CHECKPOINTS_STEPS,    
      tpu_config=tf.contrib.tpu.TPUConfig(        
          iterations_per_loop=ITERATIONS_PER_LOOP,        
          num_shards=NUM_TPU_CORES,        
          per_host_input_for_training=tf.contrib.tpu.InputPipelineConfig.PER_HOST_V2))


IOPub data rate exceeded.
The notebook server will temporarily stop sending output
to the client in order to avoid crashing it.
To change this limit, set the config variable
`--NotebookApp.iopub_data_rate_limit`.

Current values:
NotebookApp.iopub_data_rate_limit=1000000.0 (bytes/sec)
NotebookApp.rate_limit_window=3.0 (secs)



In [0]:
# Maybe don't need this because I'm not using TFHUB
# os.environ['TFHUB_CACHE_DIR'] = OUTPUT_DIR
bert_config = modeling.BertConfig.from_json_file(CONFIG_DIR)

model_fn = run_classifier.model_fn_builder(
    bert_config=bert_config,
    num_labels=len(label_list),
    init_checkpoint=CKPT_DIR,
    learning_rate=LEARNING_RATE,
    num_train_steps=num_train_steps,
    num_warmup_steps=num_warmup_steps,
    use_tpu=False,
    use_one_hot_embeddings=True
     )
# use_one_hot_embeddings is set as same value as use_tpu

estimator = tf.contrib.tpu.TPUEstimator(
  use_tpu=False,
  model_fn=model_fn,
  config=get_run_config(OUTPUT_DIR),
  train_batch_size=TRAIN_BATCH_SIZE,
  eval_batch_size=EVAL_BATCH_SIZE,
  predict_batch_size=PREDICT_BATCH_SIZE,
)

INFO:tensorflow:Using config: {'_model_dir': 'gs://mbertfinetune/bert-adapter-tfhub/models/korean_sa_4', '_tf_random_seed': None, '_save_summary_steps': 100, '_save_checkpoints_steps': 1000, '_save_checkpoints_secs': None, '_session_config': allow_soft_placement: true
cluster_def {
  job {
    name: "worker"
    tasks {
      key: 0
      value: "10.114.153.170:8470"
    }
  }
}
isolate_session_state: true
, '_keep_checkpoint_max': 5, '_keep_checkpoint_every_n_hours': 10000, '_log_step_count_steps': None, '_train_distribute': None, '_device_fn': None, '_protocol': None, '_eval_distribute': None, '_experimental_distribute': None, '_experimental_max_worker_delay_secs': None, '_session_creation_timeout_secs': 7200, '_service': None, '_cluster_spec': <tensorflow.python.training.server_lib.ClusterSpec object at 0x7fbca4ee7860>, '_task_type': 'worker', '_task_id': 0, '_global_id_in_cluster': 0, '_master': 'grpc://10.114.153.170:8470', '_evaluation_master': 'grpc://10.114.153.170:8470', '_is_

INFO:tensorflow:Using config: {'_model_dir': 'gs://mbertfinetune/bert-adapter-tfhub/models/korean_sa_4', '_tf_random_seed': None, '_save_summary_steps': 100, '_save_checkpoints_steps': 1000, '_save_checkpoints_secs': None, '_session_config': allow_soft_placement: true
cluster_def {
  job {
    name: "worker"
    tasks {
      key: 0
      value: "10.114.153.170:8470"
    }
  }
}
isolate_session_state: true
, '_keep_checkpoint_max': 5, '_keep_checkpoint_every_n_hours': 10000, '_log_step_count_steps': None, '_train_distribute': None, '_device_fn': None, '_protocol': None, '_eval_distribute': None, '_experimental_distribute': None, '_experimental_max_worker_delay_secs': None, '_session_creation_timeout_secs': 7200, '_service': None, '_cluster_spec': <tensorflow.python.training.server_lib.ClusterSpec object at 0x7fbca4ee7860>, '_task_type': 'worker', '_task_id': 0, '_global_id_in_cluster': 0, '_master': 'grpc://10.114.153.170:8470', '_evaluation_master': 'grpc://10.114.153.170:8470', '_is_

INFO:tensorflow:_TPUContext: eval_on_tpu True


INFO:tensorflow:_TPUContext: eval_on_tpu True


In [0]:
def model_train(estimator):
  print('Please wait...')
  # We'll set sequences to be at most 128 tokens long.
  train_features = run_classifier.convert_examples_to_features(
      train_examples, label_list, MAX_SEQ_LENGTH, tokenizer)
  print('***** Started training at {} *****'.format(datetime.datetime.now()))
  print('  Num examples = {}'.format(len(train_examples)))
  print('  Batch size = {}'.format(TRAIN_BATCH_SIZE))
  tf.logging.info("  Num steps = %d", num_train_steps)
  train_input_fn = run_classifier.input_fn_builder(
      features=train_features,
      seq_length=MAX_SEQ_LENGTH,
      is_training=True,
      drop_remainder=True)
  estimator.train(input_fn=train_input_fn, max_steps=num_train_steps)
  print('***** Finished training at {} *****'.format(datetime.datetime.now()))

In [0]:
model_train(estimator)

Please wait...



INFO:tensorflow:Writing example 0 of 70374


INFO:tensorflow:Writing example 0 of 70374


INFO:tensorflow:*** Example ***


INFO:tensorflow:*** Example ***


...

INFO:tensorflow:Skipping training since max_steps has already saved.


INFO:tensorflow:Skipping training since max_steps has already saved.


INFO:tensorflow:training_loop marked as finished


INFO:tensorflow:training_loop marked as finished


***** Finished training at 2020-01-29 11:00:47.473022 *****


In [0]:
def model_eval(estimator):
  # Eval the model.
  eval_examples = processor.get_dev_examples(TASK_DATA_DIR)
  eval_features = run_classifier.convert_examples_to_features(
      eval_examples, label_list, MAX_SEQ_LENGTH, tokenizer)
  print('***** Started evaluation at {} *****'.format(datetime.datetime.now()))
  print('  Num examples = {}'.format(len(eval_examples)))
  print('  Batch size = {}'.format(EVAL_BATCH_SIZE))

  # Eval will be slightly WRONG on the TPU because it will truncate
  # the last batch.
  eval_steps = int(len(eval_examples) / EVAL_BATCH_SIZE)
  eval_input_fn = run_classifier.input_fn_builder(
      features=eval_features,
      seq_length=MAX_SEQ_LENGTH,
      is_training=False,
      drop_remainder=True)
  result = estimator.evaluate(input_fn=eval_input_fn, steps=eval_steps)
  print('***** Finished evaluation at {} *****'.format(datetime.datetime.now()))
  output_eval_file = os.path.join(OUTPUT_DIR, "eval_results.txt")
  with tf.gfile.GFile(output_eval_file, "w") as writer:
    print("***** Eval results *****")
    for key in sorted(result.keys()):
      print('  {} = {}'.format(key, str(result[key])))
      writer.write("%s = %s\n" % (key, str(result[key])))
  anger_f1 = 2*result['anger_recall']*result['anger_precision']/(result['anger_recall']+result['anger_precision'])
  happy_f1 = 2*result['happy_recall']*result['happy_precision']/(result['happy_recall']+result['happy_precision'])
  sad_f1 = 2*result['sad_recall']*result['sad_precision']/(result['sad_recall']+result['sad_precision'])
  surprised_f1 = 2*result['surprised_recall']*result['surprised_precision']/(result['surprised_recall']+result['surprised_precision'])
  print("anger_f1: %s" % anger_f1 )
  print("happy_f1: %s" % happy_f1 )
  print("sad_f1: %s" % sad_f1 )
  print("surprised_f1: %s" % surprised_f1 )
  print("***confusion matrix***")
  for i in range(4):
    for j in range(4):
      key = "[{}][{}]".format(i,j)
      print(result[key])

In [0]:
model_eval(estimator)

INFO:tensorflow:Writing example 0 of 2800


INFO:tensorflow:Writing example 0 of 2800


INFO:tensorflow:*** Example ***


INFO:tensorflow:*** Example ***


INFO:tensorflow:guid: dev-1


...

INFO:tensorflow:Saving 'checkpoint_path' summary for global step 6597: gs://mbertfinetune/bert-adapter-tfhub/models/korean_sa_4/model.ckpt-6597


INFO:tensorflow:Saving 'checkpoint_path' summary for global step 6597: gs://mbertfinetune/bert-adapter-tfhub/models/korean_sa_4/model.ckpt-6597


INFO:tensorflow:evaluation_loop marked as finished


INFO:tensorflow:evaluation_loop marked as finished


***** Finished evaluation at 2020-01-29 11:05:38.145157 *****
***** Eval results *****
  [0][0] = 499.0
  [0][1] = 61.0
  [0][2] = 78.0
  [0][3] = 62.0
  [1][0] = 40.0
  [1][1] = 515.0
  [1][2] = 90.0
  [1][3] = 55.0
  [2][0] = 17.0
  [2][1] = 48.0
  [2][2] = 568.0
  [2][3] = 67.0
  [3][0] = 35.0
  [3][1] = 59.0
  [3][2] = 118.0
  [3][3] = 488.0
  anger_precision = 0.6651054
  anger_recall = 0.81142855
  eval_accuracy = 0.7392857
  eval_loss = 0.7207256
  global_step = 6597
  happy_precision = 0.8443316
  happy_recall = 0.7128571
  loss = 0.7207256
  sad_precision = 0.75402635
  sad_recall = 0.73571426
  surprised_precision = 0.72619045
  surprised_recall = 0.69714284
anger_f1: 0.7310167602706724
happy_f1: 0.773044134576342
sad_f1: 0.7447577576556195
surprised_f1: 0.7113702395517396
***confusion matrix***
499.0
61.0
78.0
62.0
40.0
515.0
90.0
55.0
17.0
48.0
568.0
67.0
35.0
59.0
118.0
488.0
